In [1]:
import numpy as np
import pandas as pd
import requests
import scraping_class
import time
import random
import json
import tqdm

In [160]:
logfile = 'Hvorlangterder_full_log.csv'
connector = scraping_class.Connector(logfile)

In [2]:
pd.options.display.max_columns = 100

In [3]:
df = pd.read_csv('Boliga - Behandling 2 - Befolkningstal.csv')
df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,agentDisplayName,agentRegId,area,basementSize,buildYear,city,createdDate,daysForSale,domainId,energyClass,exp,floor,groupKey,guid,id,images,inWatchlist,isActive,isForeclosure,isPremiumAgent,latitude,longitude,lotSize,municipality,net,openHouse,price,priceChangePercentTotal,propertyType,rooms,selfsale,size,squaremeterPrice,street,views,zipCode,addr_full,coordinate,Kommune,Region,Type,Befolkning
0,0,0,0,NaN,24254,1,0,1767,København K,2019-05-22T00:35:07.000Z,93,944,-,16872,3.0,NaN,147A17D5-24CA-4788-8009-F53DEFF2836C,1559151,"[{'id': 1559151, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67996,12.58478,0,101,67026,NaN,18750000,0,3,6.0,False,280,66964.0,"Kongens Nytorv 21, 3. tv",557,1050,"Kongens Nytorv 21, 3. tv København K","55.679959999999994, 12.58478",København,Region Hovedstaden,Ejerlejlighed,626508
1,1,1,1,NaN,207,1,88,1880,København K,2019-06-14T03:27:01.490Z,70,7,C,5702,NaN,1.0,EE6F79CE-E04F-4D59-9845-E08F98251FC9,1567291,"[{'id': 1567291, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67926,12.59149,0,101,0,NaN,10500000,0,3,4.0,False,83,126506.0,"Nyhavn 38, st. Dør/lejl. 4",139,1051,"Nyhavn 38, st. Dør/lejl. 4 København K","55.67926, 12.59149",København,Region Hovedstaden,Ejerlejlighed,626508
2,2,2,2,NaN,117,1,0,1755,København K,2019-04-24T00:09:51.880Z,121,171,C,9486,1.0,NaN,23F2A968-9A3B-44C0-BB55-22A9005DEC2C,1549766,"[{'id': 1549766, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67981,12.59206,0,101,0,NaN,15000000,0,3,7.0,False,274,54744.0,"Nyhavn 53, 1.",110,1051,"Nyhavn 53, 1. København K","55.679809999999996, 12.59206",København,Region Hovedstaden,Ejerlejlighed,626508
3,3,3,3,NaN,17151,1,0,1880,København K,2019-05-08T00:04:30.737Z,107,91,C,17609,5.0,1.0,0CB9E79E-C298-47B9-88E4-23F67F0A0A5B,1554276,"[{'id': 1554276, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67926,12.59149,0,101,70224,2019-08-25T14:00:00.000Z,19995000,0,3,7.0,False,296,67550.0,"Nyhavn 38, 5. tv.",317,1051,"Nyhavn 38, 5. tv. København K","55.67926, 12.59149",København,Region Hovedstaden,Ejerlejlighed,626508
4,4,4,4,NaN,17489,1,0,1873,København K,2019-07-12T03:21:26.060Z,42,9,C,5590,3.0,NaN,A7380CBF-7220-447E-878E-5B044910369E,1576691,"[{'id': 1576691, 'date': '2019-08-23T14:46:45....",False,True,False,False,55.67852,12.59114,0,101,0,NaN,8750000,0,3,7.0,False,163,53680.0,"Herluf Trolles Gade 22, 3. th",122,1052,"Herluf Trolles Gade 22, 3. th København K","55.67851999999999, 12.59114",København,Region Hovedstaden,Ejerlejlighed,626508


In [4]:
df.shape

(65559, 45)

In [164]:
def create_url_and_scrape(lat_long):
    url = 'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?poitypes='
    poitypes = ["lake", "forest", "doctor", "supermarket", "school", "daycare", "hospital", "metro",
                "train", "stop", "strain", "junction", "pharmacy", "library", "coast", "airport"]
    coor = lat_long.split(' ')
    coordinates = '&fromcoord=' + ''.join(coor)
    auth = '&mot=foot&token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInByaXZzIjogInIxWjByMEYwazZCdFdxUWNPVXlrQi95NlNVcEp2MlFiZ3lYZXRxNEhZNFhPLzNZclcwK0s5dz09In0.fP4JWis69HmaSg5jVHiK8nemiCu6VaMULSGGJyK4D4PkWq4iA1+nSHWMaHxepKwJ83sEiy9nMNZhv7BcktRNrA'
    tempurl = url + ','.join(poitypes) + coordinates + auth
    time.sleep(0.3)
    print(1)
    response = requests.get(tempurl)
    js = response.json()
    data = dict()
    data['lake'] = js['lake']['routedmeters'] 
    data['forest'] = js['forest']['routedmeters'] 
    data['doctor'] = js['doctor']['routedmeters']
    data['supermarket'] = js['supermarket']['routedmeters']
    data['school'] = js['school']['routedmeters']
    data['daycare'] = js['daycare']['routedmeters']
    data['hospital'] = js['hospital']['routedmeters']
    data['metro'] = js['metro']['routedmeters']
    data['train'] = js['train']['routedmeters']
    data['strain'] =js['strain']['routedmeters']
    data['junction'] = js['junction']['routedmeters']
    data['pharmacy'] = js['pharmacy']['routedmeters']
    data['library'] = js['library']['routedmeters']
    data['coast'] = js['coast']['routedmeters']
    data['airport'] = js['airport']['routedmeters']
    return data
    

In [131]:
create_url_and_scrape('55.679959999999994,12.58478')

1


{'airport': 8078.37,
 'coast': 413.79,
 'daycare': 394.28,
 'doctor': 111.11,
 'forest': 7227.33,
 'hospital': 2673.32,
 'junction': 4656.46,
 'lake': 1132.04,
 'library': 927.71,
 'metro': 106.28,
 'pharmacy': 386.65,
 'school': 350.05,
 'strain': 1056.5,
 'supermarket': 240.96,
 'train': 1056.5}

In [132]:
df_test = df.sample(10).copy()

In [133]:
df_test.head()

,Unnamed: 0,Unnamed: 0.1,agentDisplayName,agentRegId,area,basementSize,buildYear,city,createdDate,daysForSale,domainId,energyClass,exp,floor,groupKey,guid,id,images,inWatchlist,isActive,isForeclosure,isPremiumAgent,latitude,longitude,lotSize,municipality,net,openHouse,price,priceChangePercentTotal,propertyType,rooms,selfsale,size,squaremeterPrice,street,views,zipCode,addr_full,coordinate
39684,39684,34,NaN,296,10,0,1907,Tørring,2018-06-12T04:14:43.360Z,437,7,G,1046,NaN,NaN,665C1F8D-B4CB-475E-A1A8-677A90A92A7D,1452116,"[{'id': 1452116, 'date': '2019-08-23T15:06:40....",False,True,False,False,55.81466,9.53030,1064,766,0,NaN,675000,-9,1,5.0,False,173,3901.0,Stenagervej 17,1,7160,Stenagervej 17 Tørring,"55.814659999999996, 9.5303"
32675,32675,25,NaN,740,8,0,0,Gram,2019-05-14T03:52:52.803Z,101,6,-,336,NaN,NaN,NaN,1556757,"[{'id': 1556757, 'date': '2019-08-23T15:02:56....",False,True,False,False,55.31433,8.97682,1140,510,0,NaN,45600,0,7,0.0,False,0,0.0,Ållingvej 8,0,6510,Ållingvej 8 Gram,"55.314330000000005, 8.97682"
14941,14941,41,NaN,18892,6,0,1981,Gørlev,2019-05-29T03:32:46.700Z,86,6,-,1571,NaN,NaN,982C339B-2DC0-40A5-B009-429721082BEA,1562059,"[{'id': 1562059, 'date': '2019-08-23T14:54:00....",False,True,False,False,55.53116,11.10163,1380,326,0,NaN,795000,0,4,3.0,False,72,11041.0,Blishønevej 6,3,4281,Blishønevej 6 Gørlev,"55.53116, 11.10163"
9040,9040,40,NaN,165,3,0,1971,Vejby,2019-04-14T03:39:30.410Z,130,11,-,1799,NaN,NaN,CC9C9752-B7FE-46B4-8C70-DEC4D76DFEFE,1546206,"[{'id': 1546206, 'date': '2019-08-23T14:51:05....",False,True,False,False,56.07179,12.11324,1752,270,5712,NaN,1598000,-11,4,4.0,False,131,12198.0,"Nannasvej 17, Holløselund",64,3210,"Nannasvej 17, Holløselund Vejby","56.07179, 12.11324"
39943,39943,43,NaN,24,8,0,0,Grindsted,2009-05-27T05:02:16.620Z,3740,11,-,243,NaN,NaN,00000000-0000-0000-0000-000000000000,389505,"[{'id': 389505, 'date': '2019-08-23T15:06:48.0...",False,True,False,False,55.81205,9.02180,1516,530,729,NaN,185000,-25,7,0.0,False,0,0.0,"Kirkehusvej 40, Filskov",0,7200,"Kirkehusvej 40, Filskov Grindsted","55.81205, 9.0218"


In [134]:
out = df_test['coordinate'].apply(create_url_and_scrape)

1
1
1
1
1
1
1
1
1
1


In [135]:
for i in out:
    print(len(i))

15
15
15
15
15
15
15
15
15
15


In [136]:
d = pd.DataFrame(out)
d

,coordinate
39684,"{'lake': 4115.23, 'forest': 425.21, 'doctor': ..."
32675,"{'lake': 6078.42, 'forest': 1910.74, 'doctor':..."
14941,"{'lake': 7046.1, 'forest': 8963.81, 'doctor': ..."
9040,"{'lake': 945.6, 'forest': 3459.89, 'doctor': 1..."
39943,"{'lake': 6101.25, 'forest': 3264.64, 'doctor':..."
45212,"{'lake': 3963.34, 'forest': 4047.77, 'doctor':..."
19264,"{'lake': 3585.0, 'forest': 2074.34, 'doctor': ..."
55792,"{'lake': 5039.08, 'forest': 1790.69, 'doctor':..."
34386,"{'lake': 4799.65, 'forest': 480.56, 'doctor': ..."
2709,"{'lake': 1396.74, 'forest': 5880.26, 'doctor':..."


In [138]:
e = df_test.join(d, rsuffix='hvorlangt')

In [156]:
def get_lakekey(key_column):
    return key_column['lake']

In [157]:
o = e['coordinatehvorlangt'].apply(get_key)

In [158]:
o

39684    4115.23
32675    6078.42
14941    7046.10
9040      945.60
39943    6101.25
45212    3963.34
19264    3585.00
55792    5039.08
34386    4799.65
2709     1396.74
Name: coordinatehvorlangt, dtype: float64

In [165]:
# Due to a jupyter notebook uncertainty and an estimated running time of 18 hours, this line was run in Visual Studio Code
#df['hvorlangt_total'] = df['coordinate'].apply(create_url_and_scrape)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 